In [8]:
import requests
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd

In [2]:
#product to be scrapped
searchString = 'iphone10'

# URL for product search
flipkart_url = "https://www.flipkart.com/search?q=" + searchString

# Connecting with the website and parsing the html file
uClient = uReq(flipkart_url)
flipkartPage = uClient.read()
uClient.close()
flipkart_html = bs(flipkartPage, "html.parser")

In [5]:
# Finding the class with appropriate data we want
bigboxes = flipkart_html.findAll("div", {"class": "_1AtVbE col-12-12"})

# Keeping only the required data
del bigboxes[0:2] 
box = bigboxes[0]

# redirecting to the product page
productLink = "https://www.flipkart.com" + box.div.div.div.a['href'] # extracting the actual product link
prodRes = requests.get(productLink) # getting the product
prod_html = bs(prodRes.text, "html.parser")

# Finding the class with the data we need
commentboxes = prod_html.find_all('div', {'class':
"_16PBlm"})

In [7]:
# Collecting the reviews and creating a list of dictionaries of the product review
reviews = []
for commentbox in commentboxes:
    try:
        name = commentbox.div.div.find_all('p','_2sc7ZR _2V5EHH')[0].text
#         print(name)
    except:
        name = 'No'
#         print(name)
    try:
        rating = commentbox.div.div.div.div.text
    except:
        rating = 'No Rating'
    try:
        commentHead = commentbox.div.div.div.p.text
    except:
        commentHead = 'No Comment Heading'
    try:
        comtag = commentbox.div.div.find_all('div', {'class': ''})
        custComment = comtag[0].div.text
    except:
        custComment = 'No Customer Comment'
        
    mydict = {"Product": searchString, "Name": name, "Rating": rating, "CommentHead": commentHead,
                              "Comment": custComment}
    reviews.append(mydict)

In [11]:
# Getting the data into a tabular form

name = []
rating = []
comment = []
feedback = []

for x in reviews:
    name.append(x['Name'])
    rating.append(x['Rating'])
    comment.append(x['CommentHead'])
    feedback.append(x['Comment'])
    
product_feedback = pd.DataFrame({'customer':name,'rating':rating,'comment':comment,'review':feedback})

product_feedback

,customer,rating,comment,review
0,Rakesh Soni,5,Excellent,awesome screen quality better than any other s...
1,Sumit Shaw,4,Really Nice,Was never an iPhone user. But trust me this pr...
2,Fahad Mahmood,5,Perfect product!,Over all iphone x is value for money.It justif...
3,jitendra singh,5,Awesome device,First let me share my experience with Flipkart...
4,Nishikant Mhatre,5,Best iPhone ever. Killer one...,I am using iPhone since almost last 7 years an...
5,Christie Watsa,5,Excellent Phone,I highly recommend this phone if you have the ...
6,Flipkart Customer,5,Best Phone! Period.,First of all I have to thank Flipkart for prov...
7,Saurabh Chhabra,5,"Great service by Flipkart, high hopes from App...",So far so good. Have been using Android before...
8,Purav Mehta,5,Early Impressions of World's Most Awaited Devi...,"Okay, so here it is... iPhone X (it's 10 - TEN..."
9,Flipkart Customer,5,The complete phone.,Most complete phone there is. Looks Good ? Of ...
